# Análise do risco de inadimplência dos mutuários

Este projeto é para preparar um relatório para a divisão de empréstimos de um banco. É preciso descobrir se o estado civil de um cliente e o número de filhos têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

O relatório será considerado ao criar uma **pontuação de crédito** de um cliente em potencial. A **contagem de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.

Será necessário realizar um pré processamento dos dados para que estes fiquem mais limpos e concisos para postoriormente serem analisados.

Neste trabalho queremos avaliar 4 hipoteses, são elas:
- Existe uma correlação entre o nível de renda e do pagamento em dia?
- Existe uma correlação entre o status familiar e o pagamento em dia?
- Existe uma correlação entre ter filhos ou não e o pagamento em dia?
- Existe uma correlação com a finalidade do crédito afeta a taxa de inadimplência?

## Abra o arquivo de dados e veja as informações gerais.


In [1]:
# Carregando biblioteca Pandas
import pandas as pd

# Carregue os dados
try:
    default_risk = pd.read_csv('/Users/thiagodelreyramosdemelo/Desktop/Pessoal/Practicum/02 Pre Processamento de Dados Projeto 2/credit_scoring_eng.csv')
except:
    default_risk = pd.read_csv('/datasets/credit_scoring_eng.csv') 

## Tarefa 1. Exploração de dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo


In [2]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
default_risk.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# vamos exibir as primeiras 60 linhas
display(default_risk.head(60))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Os dados da coluna "experiência de trabalho em dias" (days_employed) estão inconsistentes:
- não é possível haver dias negativos de experiência de trabalho;
- e quando os dias estão positivos estão com valores irreais, pois contabilizam mais de 300 anos, 1000 anos, por exemplo. E estão coincidindo quando o tipo de renda é aposentado/"retiree".

A coluna "educação do cliente" (education) está com o preenchimento (escrita) despadronizado, dificultando assim a análise dos dados.

Quando há valor ausente na coluna "experiência de trabalho em dias" também está havendo na "renda mensal" (total_income).

A coluna propósito/"purpose" existe diversas descrições que dizem a mesma coisa, sendo necessário uma atualização das informações para um banco de dados mais limpo.

Estas situações deverão ser estudas e tratadas para que não haja impacto nos resultados das analises finais.



In [4]:
# Obter informações de valores ausentes sobre dados
display(default_risk.isna().sum())


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Este relatório possui 21525 linhas de dados com 2 colunas possuindo valores ausentes. As colunas de "experiência de trabalho em dias" (days_employed) e de "renda mensal" (total_income) há lacunas não preenchidas. Ambas possuiem 19351 linhas com dados preenchidos e 2174 com ausência de valores.

In [5]:
# Vejamos a tabela filtrada com valores ausentes na coluna 'days_employed' com dados ausentes
default_risk_filtered = (default_risk.loc[default_risk['days_employed'].isna()])
display(default_risk_filtered)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Pela amostragem listada, observamos o mesmo volume de valor ausente na coluna "experiência de trabalho em dias" e na coluna "renda mensal". Deve se analisar melhor para verificar se a ausência de valor nesta primeira coluna coicide com a ausência na segunda em 100% dos casos. 

In [6]:
# Vamos aplicar algumas condições para analisar essa padronização de valores ausentes coincidindo entre as colunas.
missing_values = default_risk.loc[(default_risk['days_employed'].isna()) | (default_risk['total_income'].isna())]
missing_values1 = default_risk.loc[(default_risk['days_employed'].isna()) & (~default_risk['total_income'].isna())]
missing_values2 = default_risk.loc[(~default_risk['days_employed'].isna()) & (default_risk['total_income'].isna())]

print(missing_values.shape[0])
print()
print(missing_values1.shape[0])
print()
print(missing_values2.shape[0])

2174

0

0


**Conclusão intermediária**

O número de linhas na tabela filtrada corresponde ao número de valores ausentes.

É visto que quando há um valor ausente na coluna "experiência de trabalho em dias" a coluna "renda mensal" também se encontra na ausência de valor.

Será verificado a significância da quantidade de insidência de valores ausentes em comparação com o total de informações que temos no DataFrame default_risk.

In [7]:
# Vamos investigar clientes que com valores ausentes na coluna "days_employed".
days_employed_total = default_risk['days_employed'].isna().sum()
record_total = default_risk['gender'].count()

days_employed_perc = (days_employed_total / record_total)*100

print(days_employed_perc)

10.099883855981417


In [8]:
# Verificar a distribuição

display(default_risk_filtered['income_type'].value_counts())
print()



employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

Há neste banco de dados, aproximadamente, **10,1%** de valores ausentes nas colunas "experiência de trabalho em dias" e "renda mensal". É um volume bem significativo de dados.

Estes valores estão distribuídos da seguinte forma, considerando o tipo de renda dos clientes:
- 1105 funcionário;
- 508 negócio;
- 413 aposentados;
- 147 funcionário público;
- 1 empreendedor.

**Possíveis motivos para valores ausentes nos dados**

Esta incoscistência tem como possíveis motivos os seguintes:
- Pessoas que nunca trabalharam;
- Trabalharam, mas não tiveram como comprovar por não ter sido registrado formalmente;
- Não declaração de tais informações;
- Problemas técnico.

Estas faltas de informações ocorrem nas seguintes colunas:
- "experiência de trabalho em dias"/"days_employed";
- "renda mensal"/"total_income").

In [9]:
# Verificar a distribuição em todo o conjunto de dados
display(default_risk)
print()
display(missing_values)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Conclusão intermediária**

A distribuição no conjunto de dados original é semelhante à distribuição da tabela filtrada podendo ser observado um único padrão de quando:
- quando não há informação na coluna de dias de experiência também falta na coluna referente à renda mensal.

Este padrão foi verificado filtrando somente as linhas que possuem valor ausente numa das colunas e comparado com as informações da outra.

Estes valores ausentes serão abordados posteriormente, pois durante as transformações de dados necessários poderemos conseguir identificar algum padrão que não foi possível observar neste momento. Caso isso ocorra, ações serão tomadas para uma análise mais limpa dos dados.

Serão analisados os dados das demais colunas e temos como próximos passos os seguintes:

- análise dos dados se há duplicatas;
- análise de dados problemáticos;
- preenchimento dos valores ausentes;
- categorização dos dados.

## Transformação de dados


In [10]:
# Vamos ver todos os valores na coluna de educação para verificar se há grafias que precisarão ser corrigidas
display(sorted(default_risk['education'].unique()))

print(default_risk.loc[default_risk['education']=='graduate degree'])

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

       children  days_employed  dob_years        education  education_id  \
6551          0   -5352.038180         58  graduate degree             4   
12021         3   -5968.075884         36  graduate degree             4   
12786         0  376276.219531         62  graduate degree             4   
21519         1   -2351.431934         37  graduate degree             4   

      family_status  family_status_id gender    income_type  debt  \
6551        married                 0      M       employee     0   
12021       married                 0      F  civil servant     0   
12786       married                 0      F        retiree     0   
21519      divorced                 3      M       employee     0   

       total_income                      purpose  
6551      42945.794          going to university  
12021     17822.757        purchase of the house  
12786     40868.031  buy residential real estate  
21519     18551.846   buy commercial real estate  


Vamos passar as grafias para letras minúsculas para termos registros mais concisos e vamos alterar a opção 'graduate degree' para 'bachelor\'s degree'.

Com isso precisaremos atualizar os dados referentes na coluna 'education_id'.

Esta ação é necessária para uma melhor análise e categorização de algumas informações.

In [11]:
# Corrigindo os registros das colunas 'education' e 'education_id'
default_risk['education'] = default_risk['education'].str.lower()
default_risk['education'] = default_risk['education'].replace('graduate degree', 'bachelor\'s degree')
default_risk['education_id'] = default_risk['education_id'].replace(4,0)

In [12]:
# Verificando todos os valores na coluna para ter certeza de que os corrigimos
display(sorted(default_risk['education'].unique()))
print()
print(default_risk['education'].value_counts())
print()
print(default_risk.loc[default_risk['education']=='graduate degree'])

["bachelor's degree",
 'primary education',
 'secondary education',
 'some college']


secondary education    15233
bachelor's degree       5266
some college             744
primary education        282
Name: education, dtype: int64

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


Agora vamos analisar os dados da coluna `purpose`.

In [13]:
# Vamos ver a distribuição de valores na coluna `purpose`
print(default_risk['purpose'].isna().sum())
print()
print(sorted(default_risk['purpose'].unique()))
print()
print(default_risk['purpose'].value_counts())

0

['building a property', 'building a real estate', 'buy commercial real estate', 'buy real estate', 'buy residential real estate', 'buying a second-hand car', 'buying my own car', 'buying property for renting out', 'car', 'car purchase', 'cars', 'construction of own property', 'education', 'getting an education', 'getting higher education', 'going to university', 'having a wedding', 'housing', 'housing renovation', 'housing transactions', 'profile education', 'property', 'purchase of a car', 'purchase of my own house', 'purchase of the house', 'purchase of the house for my family', 'real estate transactions', 'second-hand car purchase', 'supplementary education', 'to become educated', 'to buy a car', 'to get a supplementary education', 'to have a wedding', 'to own a car', 'transactions with commercial real estate', 'transactions with my real estate', 'university education', 'wedding ceremony']

wedding ceremony                            797
having a wedding                          

Nesta coluna de propósito possui diversas informações com escritas diferentes, mas que nos dizem a mesma coisa. Com isso e gerado opções desnecessárias.

Vamos juntar essas semelhanças em apenas 8 (oito) opções para deixar a análise mais limpa.

Os itens são:
- building a property (undefined);
- buy a car;
- buy commercial real estate;
- buy or renovate residential properties;
- buy real estate (undefined);
- construction of own property;
- education;
- wedding.

In [14]:
#corrigindo os dados

default_risk['purpose'] = default_risk['purpose'].replace\
            (['building a property', 'building a real estate'], 'building a property (undefined)')
default_risk['purpose'] = default_risk['purpose'].replace\
            (['buy real estate', 'buying property for renting out',\
              'real estate transactions', 'housing transactions', 'property'], 'buy real estate (undefined)')
default_risk['purpose'] = default_risk['purpose'].replace\
            ('transactions with commercial real estate', 'buy commercial real estate')
default_risk['purpose'] = default_risk['purpose'].replace\
            (['buy residential real estate', 'purchase of my own house', 'purchase of the house',\
              'purchase of the house for my family', 'housing', 'transactions with my real estate',\
              'housing renovation'], 'buy or renovate residential properties')
default_risk['purpose'] = default_risk['purpose'].replace\
            (['buying my own car', 'car', 'cars', 'car purchase', 'purchase of a car', 'to buy a car',\
              'to own a car', 'buying a second-hand car', 'second-hand car purchase'], 'buy a car')
default_risk['purpose'] = default_risk['purpose'].replace\
            (['getting an education', 'supplementary education', 'to become educated',\
              'to get a supplementary education', 'university education', 'getting higher education',\
              'going to university', 'profile education'], 'education')
default_risk['purpose'] = default_risk['purpose'].replace\
            (['having a wedding', 'to have a wedding', 'wedding ceremony'], 'wedding')


In [15]:
# Verificando a coluna `purpose` novamente para ter certeza de que está tudo corrigido
print(sorted(default_risk['purpose'].unique()))
print()
print(default_risk['purpose'].value_counts())
print()


['building a property (undefined)', 'buy a car', 'buy commercial real estate', 'buy or renovate residential properties', 'buy real estate (undefined)', 'construction of own property', 'education', 'wedding']

buy or renovate residential properties    4404
buy a car                                 4315
education                                 4022
buy real estate (undefined)               3240
wedding                                   2348
buy commercial real estate                1315
building a property (undefined)           1246
construction of own property               635
Name: purpose, dtype: int64



Verificando os dados na coluna `children`

In [16]:
# Vamos ver a distribuição de valores na coluna `children`
print(sorted(default_risk['children'].unique()))
print()
print(default_risk['children'].value_counts())
print()

values_total = default_risk['children'].count()

print(f'Total de dados na coluna = {values_total}.')
print()

values_one_neg = default_risk[default_risk['children'] == -1]['children'].count()
print(f'Total de valores \"-1\" na coluna = {values_one_neg}.')
prob_one_neg_total = (values_one_neg/values_total)
print(f'Total de valores \"-1\" nesta coluna = {prob_one_neg_total:.2%}.')

print()

values_twenty = default_risk[default_risk['children'] == 20]['children'].count()
print(f'Total de valores \"20\" na coluna = {values_twenty}.')
prob_twenty_total = (values_twenty/values_total)
print(f'Total de valores \"20\" nesta coluna = {prob_twenty_total:.2%}.')

[-1, 0, 1, 2, 3, 4, 5, 20]

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Total de dados na coluna = 21525.

Total de valores "-1" na coluna = 47.
Total de valores "-1" nesta coluna = 0.22%.

Total de valores "20" na coluna = 76.
Total de valores "20" nesta coluna = 0.35%.


A coluna "children" possui 47 informações do número 1 (um) como sinal de negativo (-1), sendo este equivalente à 0,22% do total não tendo um impacto considerável resultado final.

Há também, nesta coluna, 76 informações como "20" filhos e estes lançamentos são equivalentes à 0,35% do total.

Estes dados problemáticos provavelmente foram ocorridos por erro de digitação durante o lançamento das informações. Estes serão alterados/corrigidos da seguinte forma:
- os valores "-1" serão alterados para "1" (um);
- os valores "20" serão alterados para "2" (dois).

In [17]:
#Corrija os dados da coluna 'children'

default_risk['children'] = default_risk['children'].replace(-1, 1)
default_risk['children'] = default_risk['children'].replace(20, 2)


In [18]:
# Verificando a coluna `children` novamente para ter certeza de que está tudo corrigido
print(sorted(default_risk['children'].unique()))
print()
print(default_risk['children'].value_counts())
print()


[0, 1, 2, 3, 4, 5]

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64



Verificando os dados na coluna `days_employed`.

In [19]:
# Encontrando possíveis dados problemáticos em `days_employed`.
print(default_risk['days_employed'].value_counts())
print()

#Verificando a média dos valores negativos
print(default_risk[default_risk['days_employed'] <=0].mean())
print()

#contando quantidade de valores negativos
print(default_risk[default_risk['days_employed'] <=0].count())
print()

#Verificando se os valores positivos
print(default_risk[default_risk['days_employed'] > 0].count())
print()
print(default_risk[default_risk['days_employed'] > 20000].count())

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

children                0.562744
days_employed       -2353.015932
dob_years              39.818245
education_id            0.797372
family_status_id        0.969634
debt                    0.087326
total_income        27837.509634
dtype: float64

children            15906
days_employed       15906
dob_years           15906
education           15906
education_id        15906
family_status       15906
family_status_id    15906
gender              15906
income_type         15906
debt                15906
total_income        15906
purpose             15906
dtype: int64

children            3445
days_employed       3445
dob_years           3445
education           3445
education_id        3445
family_status       3445
family_status_id    3445
gend

Foi observado, anteriormente, que os valores positivos para dias trabalhados coincidem com o tipo de renda de pessoas aposentadas e nesta última análise foi verificado que 100% destes estão com valores irreais, pois estão contabilizando 300, 500, 1000 anos de trabalho.

Partindo do princípio que 62 anos é uma idade média em que as pessoas se aposentam, estes valores elevados de dias trabalhados serão substituídos por 22630 que equivale à estes 62 anos citados.

Há, também, muitos valores negativos e não fazem sentido, pois não é possível ter dias negativos trabalhados. Provavelmente foi um erro técnico que ocasionou essas linhas problemáticas. Estes equivalem à 82,19% do total das informações e estes sinais negativos serão retirados dos dados.

In [20]:
#Corrigindo os valores extremamente altos de dias trabalhados para o valor mediano das pessoas com mais de 45 anos
def calc1(days_employed_high):
    new_days = 22630
    if days_employed_high > 0:
        return new_days
    else:
        return days_employed_high

try:
    default_risk['days_employed'] = default_risk['days_employed'].apply(calc1)
except:
    'Erro de execução!'


#Corrigindo os valores com sinal negativo
i = -1
def calc2(days_employed_neg):
    days = i * days_employed_neg
    if days > 0:
        return days
    else:
        return days_employed_neg
    
try:
    default_risk['days_employed'] = default_risk['days_employed'].apply(calc2)
except:
    'Erro de execução!'


print(default_risk.head(10))


   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42    bachelor's degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0   22630.000000         53  secondary education             1   
5         0     926.185831         27    bachelor's degree             0   
6         0    2879.202052         43    bachelor's degree             0   
7         0     152.779569         50  secondary education             1   
8         2    6929.865299         35    bachelor's degree             0   
9         0    2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

In [21]:
# Verificando o resultado para certificar de que foram corrigidos
print(default_risk[default_risk['days_employed'] <=0].count())
print()
print(default_risk[default_risk['days_employed'] > 0].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

children            19351
days_employed       19351
dob_years           19351
education           19351
education_id        19351
family_status       19351
family_status_id    19351
gender              19351
income_type         19351
debt                19351
total_income        19351
purpose             19351
dtype: int64


Vamos verificar agora os dados sobre as idades dos clientes (coluna `'dob_years'`)

In [22]:
# Verificando `dob_years` para valores suspeitos
print(sorted(default_risk['dob_years'].unique()))
print()
print(default_risk['dob_years'].value_counts())
print()

values_dob_years_total = default_risk['dob_years'].count()
print(f'Total de dados na coluna = {values_dob_years_total}.')
print()

values_dob_years_zero = default_risk[default_risk['dob_years'] == 0]['dob_years'].count()
print(f'Total de valores \"0\" na coluna = {values_dob_years_zero}.')
prob_dob_years_zero = (values_dob_years_zero/values_dob_years_total)
print(f'Total de valores \"0\" nesta coluna = {prob_dob_years_zero:.2%}.')


[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Total de dados na coluna = 21525.

Total de valores "0" na coluna = 101.
Total de valores "0" nesta coluna = 0.47%.


São observados 101 valores de idades do cliente como "0" (zero). Pode ter sido por erro de digitação, mas não tem como saber quais idades as pessoas queriam ter lançado. Estas informações correspondem à **0,47%** do total e serão substituídas.

In [23]:
# Verificando a média e a mediana das idades.
value_dob_years_mean = (int(default_risk[default_risk['dob_years'] > 0]['dob_years'].mean()))
print(value_dob_years_mean)
print()
value_dob_years_median = (int(default_risk[default_risk['dob_years'] > 0]['dob_years'].median()))
print(value_dob_years_median)


43

43


Estes 101 valores de idades do cliente como "0" (zero) serão substituídos pela média das demais idades.

In [24]:
# Substituindo os valores "0" (zero) na coluna 'dob_years'.
default_risk['dob_years'] = default_risk['dob_years'].replace(0, value_dob_years_mean)

In [25]:
# Verificando o resultado se foram corrigidos.
print(default_risk['dob_years'].unique())
print()
print(default_risk['dob_years'].value_counts())


[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


Analisando as informações da coluna `family_status`.

In [26]:
# Vamos ver os valores da coluna 'family_status'.
print(sorted(default_risk['family_status'].unique()))
print()
print(default_risk['family_status'].value_counts())


['civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower']

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64


In [27]:
# Realizando um ajuste simples nesta coluna. Retirando os "espaços" do termo 'widow / widower'.
default_risk['family_status'] = default_risk['family_status'].replace('widow / widower', 'widow/widower')


In [28]:
# Verificando o resultado se foi ajustado corretamente.
print(sorted(default_risk['family_status'].unique()))
print()
print(default_risk['family_status'].value_counts())

['civil partnership', 'divorced', 'married', 'unmarried', 'widow/widower']

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow/widower          960
Name: family_status, dtype: int64


Agora vamos verificar a coluna `gender`.

In [29]:
# Vamos ver os valores na coluna 'gender'.
print(sorted(default_risk['gender'].unique()))
print()
print(default_risk['gender'].value_counts())

['F', 'M', 'XNA']

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [30]:
# Realizando um ajuste simples nesta coluna. Alterando o termo 'XNA' para 'F'.

default_risk['gender'] = default_risk['gender'].replace('XNA', 'F')

In [31]:
# Verificando o resultado se foi ajustado corretamente.

print(sorted(default_risk['gender'].unique()))
print()
print(default_risk['gender'].value_counts())

['F', 'M']

F    14237
M     7288
Name: gender, dtype: int64


Neste ponto iremos verificar a coluna `income_type`.

In [32]:
# Verificando informações na coluna 'income_type'.
print(sorted(default_risk['income_type'].unique()))
print()
print(default_risk['income_type'].value_counts())

['business', 'civil servant', 'employee', 'entrepreneur', 'paternity / maternity leave', 'retiree', 'student', 'unemployed']

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64


In [33]:
# Realizando um ajuste simples nesta coluna. Alterando o termo 'paternity / maternity leave'
# para paternity/maternity leave' (sem os "espaços" antes e depois da "barra"(/).
default_risk['income_type'] = default_risk['income_type'].replace('paternity / maternity leave', 'paternity/maternity leave')


In [34]:
# Verifique o resultado se foi corrigido.
print(sorted(default_risk['income_type'].unique()))
print()
print(default_risk['income_type'].value_counts())


['business', 'civil servant', 'employee', 'entrepreneur', 'paternity/maternity leave', 'retiree', 'student', 'unemployed']

employee                     11119
business                      5085
retiree                       3856
civil servant                 1459
entrepreneur                     2
unemployed                       2
paternity/maternity leave        1
student                          1
Name: income_type, dtype: int64


Neste ponto iremos analisar se temos duplicatas em nossos dados.

In [35]:
# Verificando duplicatas nos dados.

print(default_risk.duplicated().sum())


252


Foram encontradas 252 duplicatas em nos dados.

Nós não temos na planilha uma coluna individualizando as informações, porém é improvável que não sejam duplicatas reais, pois temos 2 colunas com informações com pontos flutuantes contendo algumas casas após a vírgula/ponto, são elas:
- 'days_employed';
- 'total_income'.

Como essas informações equivalem à apenas **1,17%** do total, não impactará na análise final se forem retiradas. Por isso iremos excluir essas linhas dos dados.

In [36]:
# Excluindo duplicatas
default_risk = default_risk.drop_duplicates()


In [37]:
# Verificando se ainda temos duplicatas
print(default_risk.duplicated().sum())

0


In [38]:
#Verificando o conjunto de dados após estas manipulações realizadas.

default_risk.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21273 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21273 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21273 non-null  int64  
 3   education         21273 non-null  object 
 4   education_id      21273 non-null  int64  
 5   family_status     21273 non-null  object 
 6   family_status_id  21273 non-null  int64  
 7   gender            21273 non-null  object 
 8   income_type       21273 non-null  object 
 9   debt              21273 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21273 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Neste ponto o nosso conjunto de dados se encontra mais limpo para que possamos efetuar as análises e categorizações necessárias. Tivemos ajustes em quase todas colunas e a redução de 252 linhas do total anterior.

# Trabalhando com valores ausentes

### Restaurar valores ausentes em `total_income`

Temos 2 (duas) colunas com valores ausentes neste banco de dados:
- "days_employed";
- "total_income".

Vamos analisar alguns parâmetros entre gênero, filhos, idade e estudo para tomada de desição. Serão verificadas as médias e medianas para a tomada de decisão de quais informações serão inseridas nos valores ausentes.

Será criada uma tabela sem as linhas contendo os valores ausentes para que seja possível uma análise assertiva dos dados.

In [39]:
# Criando uma tabela sem valores ausentes

new_table = default_risk.dropna(subset=['days_employed', 'total_income'])

display(new_table.head(60))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,buy or renovate residential properties
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,buy a car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,buy or renovate residential properties
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education
4,0,22630.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,buy or renovate residential properties
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,buy real estate (undefined)
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,buy or renovate residential properties


In [40]:
# Analisando valores médios de renda com base em alguns parâmetros

#Valor médio total
value_mean_total = new_table['total_income'].mean()
print(f'Valor médio de renda de todos os valores é de {value_mean_total:.2f}.')
print()

#Valor médio das mulheres
value_mean_gender_f = new_table.loc[new_table['gender'] == 'F']['total_income'].mean()
print(f'Valor médio de renda das mulheres é de {value_mean_gender_f:.2f}.')

#Valor médio dos homens
value_mean_gender_m = new_table.loc[new_table['gender'] == 'M']['total_income'].mean()
print(f'Valor médio de renda dos homens é de {value_mean_gender_m:.2f}.')
print()

#Valor médio das mulheres com menos de 40 anos
value_mean_gender_f_age_under40 = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['dob_years']<40)]['total_income'].mean()
print(f'Valor médio de renda das mulheres com menos de 40 anos é de {value_mean_gender_f_age_under40:.2f}.')

#Valor médio dos homens com menos de 40 anos
value_mean_gender_m_age_under40 = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['dob_years']<40)]['total_income'].mean()
print(f'Valor médio de renda dos homens com menos de 40 anos é de {value_mean_gender_m_age_under40:.2f}.')

#Valor médio das mulheres com mais ou igual a 40 anos
value_mean_gender_f_age_over40 = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['dob_years']>=40)]['total_income'].mean()
print(f'Valor médio de renda das mulheres com 40 ou mais anos é de {value_mean_gender_f_age_over40:.2f}.')

#Valor médio dos homens com mais ou igual a 40 anos
value_mean_gender_m_age_over40 = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['dob_years']>=40)]['total_income'].mean()
print(f'Valor médio de renda dos homens com 40 ou mais anos é de {value_mean_gender_m_age_over40:.2f}.')
print()

#Valor médio das mulheres com que possuem filhos
value_mean_gender_f_child = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['children']>0)]['total_income'].mean()
print(f'Valor médio de renda das mulheres que possuem filho(s) é de {value_mean_gender_f_child:.2f}.')

#Valor médio dos homens com que possuem filhos
value_mean_gender_m_child = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['children']>0)]['total_income'].mean()
print(f'Valor médio de renda dos homens que possuem filho(s) é de {value_mean_gender_m_child:.2f}.')
print()

#Valor médio das mulheres com curso superior/formadas
value_mean_gender_f_f = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')]['total_income'].mean()
print(f'Valor médio de renda das mulheres formadas é de {value_mean_gender_f_f:.2f}.')

#Valor médio dos homens com curso superior/formados
value_mean_gender_m_f = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')]['total_income'].mean()
print(f'Valor médio de renda dos homens formados é de {value_mean_gender_m_f:.2f}.')
print()

#Valor médio das mulheres com curso superior/formadas e que possuem filhos
value_mean_gender_f_f_child = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')\
                                       &(new_table['children']>0)]['total_income'].mean()
print(f'Valor médio de renda das mulheres formadas que possuem filho(s) é de {value_mean_gender_f_f_child:.2f}.')

#Valor médio dos homens com curso superior/formados e que possuem filhos
value_mean_gender_m_f_child = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')\
                                       &(new_table['children']>0)]['total_income'].mean()
print(f'Valor médio de renda dos homens formados que possuem filho(s) é de {value_mean_gender_m_f_child:.2f}.')
print()

#Valor médio das mulheres com curso superior/formadas, com menos de 40 anos e que possuem filhos
value_mean_gender_f_f_child_age_under40 = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')\
                                       &(new_table['dob_years']<40)\
                                       &(new_table['children']>0)]['total_income'].mean()
print(f'Valor médio de renda das mulheres formadas que possuem filho(s) com menos de 40 anos é de {value_mean_gender_f_f_child_age_under40:.2f}.')

#Valor médio dos homens com curso superior/formados, com menos de 40 anos e que possuem filhos
value_mean_gender_m_f_child_age_under40 = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')\
                                       &(new_table['dob_years']<40)\
                                       &(new_table['children']>0)]['total_income'].mean()
print(f'Valor médio de renda dos homens formados que possuem filho(s) com menos de 40 anos é de {value_mean_gender_m_f_child_age_under40:.2f}.')
print()

Valor médio de renda de todos os valores é de 26787.57.

Valor médio de renda das mulheres é de 24656.23.
Valor médio de renda dos homens é de 30907.14.

Valor médio de renda das mulheres com menos de 40 anos é de 24815.59.
Valor médio de renda dos homens com menos de 40 anos é de 31020.82.
Valor médio de renda das mulheres com 40 ou mais anos é de 24560.69.
Valor médio de renda dos homens com 40 ou mais anos é de 30799.25.

Valor médio de renda das mulheres que possuem filho(s) é de 24587.20.
Valor médio de renda dos homens que possuem filho(s) é de 32423.54.

Valor médio de renda das mulheres formadas é de 30305.84.
Valor médio de renda dos homens formados é de 38950.76.

Valor médio de renda das mulheres formadas que possuem filho(s) é de 29300.42.
Valor médio de renda dos homens formados que possuem filho(s) é de 41720.44.

Valor médio de renda das mulheres formadas que possuem filho(s) com menos de 40 anos é de 28349.18.
Valor médio de renda dos homens formados que possuem filho(s

In [41]:
# Analisando valores medianos de renda com base em alguns parâmetros

#Valor mediano total
value_median_total = new_table['total_income'].median()
print(f'Valor mediano de renda de todos os valores é de {value_median_total:.2f}.')
print()

#Valor mediano das mulheres
value_median_gender_f = new_table.loc[new_table['gender'] == 'F']['total_income'].median()
print(f'Valor mediano de renda das mulheres é de {value_median_gender_f:.2f}.')

#Valor mediano dos homens
value_median_gender_m = new_table.loc[new_table['gender'] == 'M']['total_income'].median()
print(f'Valor mediano de renda dos homens é de {value_median_gender_m:.2f}.')
print()

#Valor mediano das mulheres com menos de 40 anos
value_median_gender_f_age_under40 = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['dob_years']<40)]['total_income'].median()
print(f'Valor mediano de renda das mulheres com menos de 40 anos é de {value_median_gender_f_age_under40:.2f}.')

#Valor mediano dos homens com menos de 40 anos
value_median_gender_m_age_under40 = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['dob_years']<40)]['total_income'].median()
print(f'Valor mediano de renda dos homens com menos de 40 anos é de {value_median_gender_m_age_under40:.2f}.')

#Valor mediano das mulheres com mais ou igual a 40 anos
value_median_gender_f_age_over40 = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['dob_years']>=40)]['total_income'].median()
print(f'Valor mediano de renda das mulheres com 40 ou mais anos é de {value_median_gender_f_age_over40:.2f}.')

#Valor mediano dos homens com mais ou igual a 40 anos
value_median_gender_m_age_over40 = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['dob_years']>=40)]['total_income'].median()
print(f'Valor mediano de renda dos homens com 40 ou mais anos é de {value_median_gender_m_age_over40:.2f}.')
print()

#Valor mediano das mulheres com que possuem filhos
value_median_gender_f_child = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['children']>0)]['total_income'].median()
print(f'Valor mediano de renda das mulheres que possuem filho(s) é de {value_median_gender_f_child:.2f}.')

#Valor mediano dos homens com que possuem filhos
value_median_gender_m_child = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['children']>0)]['total_income'].median()
print(f'Valor mediano de renda dos homens que possuem filho(s) é de {value_median_gender_m_child:.2f}.')
print()

#Valor mediano das mulheres com curso superior/formadas
value_median_gender_f_f = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')]['total_income'].median()
print(f'Valor mediano de renda das mulheres formadas é de {value_median_gender_f_f:.2f}.')

#Valor mediano dos homens com curso superior/formados
value_median_gender_m_f = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')]['total_income'].median()
print(f'Valor mediano de renda dos homens formados é de {value_median_gender_m_f:.2f}.')
print()

#Valor mediano das mulheres com curso superior/formadas e que possuem filhos
value_median_gender_f_f_child = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')\
                                       &(new_table['children']>0)]['total_income'].median()
print(f'Valor mediano de renda das mulheres formadas que possuem filho(s) é de {value_median_gender_f_f_child:.2f}.')

#Valor mediano dos homens com curso superior/formados e que possuem filhos
value_median_gender_m_f_child = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')\
                                       &(new_table['children']>0)]['total_income'].median()
print(f'Valor mediano de renda dos homens formados que possuem filho(s) é de {value_median_gender_m_f_child:.2f}.')
print()

#Valor mediano das mulheres com curso superior/formadas, com menos de 40 anos e que possuem filhos
value_median_gender_f_f_child_age_under40 = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')\
                                       &(new_table['dob_years']<40)\
                                       &(new_table['children']>0)]['total_income'].median()
print(f'Valor mediano de renda das mulheres formadas que possuem filho(s) com menos de 40 anos é de {value_median_gender_f_f_child_age_under40:.2f}.')

#Valor mediano dos homens com curso superior/formados, com menos de 40 anos e que possuem filhos
value_median_gender_m_f_child_age_under40 = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')\
                                       &(new_table['dob_years']<40)\
                                       &(new_table['children']>0)]['total_income'].median()
print(f'Valor mediano de renda dos homens formados que possuem filho(s) com menos de 40 anos é de {value_median_gender_m_f_child_age_under40:.2f}.')
print()

Valor mediano de renda de todos os valores é de 23202.87.

Valor mediano de renda das mulheres é de 21465.17.
Valor mediano de renda dos homens é de 26834.30.

Valor mediano de renda das mulheres com menos de 40 anos é de 21678.97.
Valor mediano de renda dos homens com menos de 40 anos é de 27269.98.
Valor mediano de renda das mulheres com 40 ou mais anos é de 21327.87.
Valor mediano de renda dos homens com 40 ou mais anos é de 26595.15.

Valor mediano de renda das mulheres que possuem filho(s) é de 21481.19.
Valor mediano de renda dos homens que possuem filho(s) é de 27994.07.

Valor mediano de renda das mulheres formadas é de 26063.47.
Valor mediano de renda dos homens formados é de 32623.49.

Valor mediano de renda das mulheres formadas que possuem filho(s) é de 24387.33.
Valor mediano de renda dos homens formados que possuem filho(s) é de 33907.39.

Valor mediano de renda das mulheres formadas que possuem filho(s) com menos de 40 anos é de 23685.43.
Valor mediano de renda dos homen

Diante das análises realizadas, valores ausentes serão subistiuídos por medianas da seguintes formas:
- para pessoas que possuem nível superior (formadas), os valores ausentes serão preenchidos pela mediana referente à renda por gênero e formados.
- para pessoas que não possuem nível superior, os valores ausentes serão preenchidos pela mediana referente referente à renda por gênero.

In [42]:
#Definindo uma função para preencher os valores ausentes.
def fill_missing_values_income(row):
    total_income = row['total_income']
    gender = row['gender']
    education_id = row['education_id']
    
    if total_income > 0:
        return total_income
    if education_id == 0:
        if gender == 'M':
            return value_median_gender_m_f
        else:
            return value_median_gender_f_f
    if education_id > 0:
        if gender == 'M':
            return value_median_gender_m
        else:
            return value_median_gender_f


In [43]:
# Verificando se a função funciona.
row_values = [[0, 'M', 3], [0, 'F', 7], [10,'F',0],[0, 'M', 0], [0, 'F', 0],[10, 'M', 0]]
row_columns = ['education_id', 'gender', 'total_income']
df_teste = pd.DataFrame(row_values, columns=row_columns)
print(df_teste.head(10))
print()

try:
    df_teste['total_income'] = df_teste.apply(fill_missing_values_income, axis=1)
except:
    'Erro de execução!'

print(df_teste)

   education_id gender  total_income
0             0      M             3
1             0      F             7
2            10      F             0
3             0      M             0
4             0      F             0
5            10      M             0

   education_id gender  total_income
0             0      M        3.0000
1             0      F        7.0000
2            10      F    21465.1650
3             0      M    32623.4850
4             0      F    26063.4715
5            10      M    26834.2950


In [44]:
# Aplicando a função em todas as linhas da planilha.
default_risk['total_income'] = default_risk.apply(fill_missing_values_income, axis=1)

In [45]:
# Verificando se ocorreu tudo certo.
print(default_risk['total_income'].isna().sum())
print()
print(default_risk.loc[default_risk['total_income']==0])
print()
print(default_risk.loc[default_risk['total_income']<0])


0

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [46]:
# Verificando o número de entradas na coluna. Se 'total_income' realmente foi preenchida.

default_risk.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21273 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21273 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21273 non-null  int64  
 3   education         21273 non-null  object 
 4   education_id      21273 non-null  int64  
 5   family_status     21273 non-null  object 
 6   family_status_id  21273 non-null  int64  
 7   gender            21273 non-null  object 
 8   income_type       21273 non-null  object 
 9   debt              21273 non-null  int64  
 10  total_income      21273 non-null  float64
 11  purpose           21273 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


###  Restaurar valores em `days_employed`

Vamos analisar os mesmos parâmetros que utilizamos para avaliar 'total_income' para a tomada de desição.
- gênero;
- filhos;
- idade;
- estudo.

Também serão verificadas as médias e medianas para a tomada de decisão de quais informações serão inseridas nos valores ausentes desta coluna.

In [47]:
# Analisando valores medianos de dias trabalhados com base em alguns parâmetros

#Valor mediano total
value_median_days_employed_total = new_table['days_employed'].median()
print(f'Valor mediano de dias trabalhados de todos os valores é de {value_median_days_employed_total:.2f}.')
print()

#Valor mediano das mulheres
value_median_days_employed_gender_f = new_table.loc[new_table['gender'] == 'F']['days_employed'].median()
print(f'Valor mediano de dias trabalhados das mulheres é de {value_median_days_employed_gender_f:.2f}.')

#Valor mediano dos homens
value_median_days_employed_gender_m = new_table.loc[new_table['gender'] == 'M']['days_employed'].median()
print(f'Valor mediano de dias trabalhados dos homens é de {value_median_days_employed_gender_m:.2f}.')
print()

#Valor mediano das mulheres com menos de 40 anos
value_median_days_employed_gender_f_age_under40 = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['dob_years']<40)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados das mulheres com menos de 40 anos é de {value_median_days_employed_gender_f_age_under40:.2f}.')

#Valor mediano dos homens com menos de 40 anos
value_median_days_employed_gender_m_age_under40 = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['dob_years']<40)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados dos homens com menos de 40 anos é de {value_median_days_employed_gender_m_age_under40:.2f}.')

#Valor mediano das mulheres com mais ou igual à 40 anos
value_median_days_employed_gender_f_age_over40 = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['dob_years']>=40)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados das mulheres com 40 ou mais anos é de {value_median_days_employed_gender_f_age_over40:.2f}.')

#Valor mediano dos homens com mais ou igual à 40 anos
value_median_days_employed_gender_m_age_over40 = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['dob_years']>=40)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados dos homens com 40 ou mais anos é de {value_median_days_employed_gender_m_age_over40:.2f}.')
print()

#Valor mediano das mulheres com que possuem filhos
value_median_days_employed_gender_f_child = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['children']>0)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados das mulheres que possuem filho(s) é de {value_median_days_employed_gender_f_child:.2f}.')

#Valor mediano dos homens com que possuem filhos
value_median_days_employed_gender_m_child = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['children']>0)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados dos homens que possuem filho(s) é de {value_median_days_employed_gender_m_child:.2f}.')
print()

#Valor mediano das mulheres com curso superior/formadas
value_median_days_employed_gender_f_f = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')]['days_employed'].median()
print(f'Valor mediano de dias trabalhados das mulheres formadas é de {value_median_days_employed_gender_f_f:.2f}.')

#Valor mediano dos homens com curso superior/formados
value_median_days_employed_gender_m_f = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')]['days_employed'].median()
print(f'Valor mediano de dias trabalhados dos homens formados é de {value_median_days_employed_gender_m_f:.2f}.')
print()

#Valor mediano das mulheres com curso superior/formadas e que possuem filhos
value_median_days_employed_gender_f_f_child = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')\
                                       &(new_table['children']>0)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados das mulheres formadas que possuem filho(s) é de {value_median_days_employed_gender_f_f_child:.2f}.')

#Valor mediano dos homens com curso superior/formados e que possuem filhos
value_median_days_employed_gender_m_f_child = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')\
                                       &(new_table['children']>0)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados dos homens formados que possuem filho(s) é de {value_median_days_employed_gender_m_f_child:.2f}.')
print()

#Valor mediano das mulheres com curso superior/formadas, com menos de 40 anos e que possuem filhos
value_median_days_employed_gender_f_f_child_age_under40 = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')\
                                       &(new_table['dob_years']<40)\
                                       &(new_table['children']>0)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados das mulheres formadas que possuem filho(s) com menos de 40 anos é de {value_median_days_employed_gender_f_f_child_age_under40:.2f}.')

#Valor mediano dos homens com curso superior/formados, com menos de 40 anos e que possuem filhos
value_median_days_employed_gender_m_f_child_age_under40 = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')\
                                       &(new_table['dob_years']<40)\
                                       &(new_table['children']>0)]['days_employed'].median()
print(f'Valor mediano de dias trabalhados dos homens formados que possuem filho(s) com menos de 40 anos é de {value_median_days_employed_gender_m_f_child_age_under40:.2f}.')
print()


Valor mediano de dias trabalhados de todos os valores é de 2194.22.

Valor mediano de dias trabalhados das mulheres é de 2539.76.
Valor mediano de dias trabalhados dos homens é de 1662.37.

Valor mediano de dias trabalhados das mulheres com menos de 40 anos é de 1387.33.
Valor mediano de dias trabalhados dos homens com menos de 40 anos é de 1228.64.
Valor mediano de dias trabalhados das mulheres com 40 ou mais anos é de 4662.14.
Valor mediano de dias trabalhados dos homens com 40 ou mais anos é de 2465.40.

Valor mediano de dias trabalhados das mulheres que possuem filho(s) é de 1755.00.
Valor mediano de dias trabalhados dos homens que possuem filho(s) é de 1553.83.

Valor mediano de dias trabalhados das mulheres formadas é de 2011.17.
Valor mediano de dias trabalhados dos homens formados é de 1659.31.

Valor mediano de dias trabalhados das mulheres formadas que possuem filho(s) é de 1669.09.
Valor mediano de dias trabalhados dos homens formados que possuem filho(s) é de 1403.58.

Valo

In [48]:
# Analisando valores médios de dias trabalhados com base em alguns parâmetros

#Valor médio total
value_mean_days_employed_total = new_table['days_employed'].mean()
print(f'Valor médio de dias trabalhados de todos os valores é de {value_mean_days_employed_total:.2f}.')
print()

#Valor médio das mulheres
value_mean_days_employed_gender_f = new_table.loc[new_table['gender'] == 'F']['days_employed'].mean()
print(f'Valor médio de dias trabalhados das mulheres é de {value_mean_days_employed_gender_f:.2f}.')

#Valor médio o dos homens
value_mean_days_employed_gender_m = new_table.loc[new_table['gender'] == 'M']['days_employed'].mean()
print(f'Valor médio de dias trabalhados dos homens é de {value_mean_days_employed_gender_m:.2f}.')
print()

#Valor médio das mulheres com menos de 40 anos
value_mean_days_employed_gender_f_age_under40 = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['dob_years']<40)]['days_employed'].mean()
print(f'Valor médio de dias trabalhados das mulheres com menos de 40 anos é de {value_mean_days_employed_gender_f_age_under40:.2f}.')

#Valor médio dos homens com menos de 40 anos
value_mean_days_employed_gender_m_age_under40 = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['dob_years']<40)]['days_employed'].mean()
print(f'Valor médio de dias trabalhados dos homens com menos de 40 anos é de {value_mean_days_employed_gender_m_age_under40:.2f}.')

#Valor médio das mulheres com mais de 40 anos
value_mean_days_employed_gender_f_age_over40 = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['dob_years']>=40)]['days_employed'].mean()
print(f'Valor médio de dias trabalhados das mulheres com 40 ou mais anos é de {value_mean_days_employed_gender_f_age_over40:.2f}.')

#Valor médio dos homens com mais de 40 anos
value_mean_days_employed_gender_m_age_over40 = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['dob_years']>=40)]['days_employed'].mean()
print(f'Valor médio de dias trabalhados dos homens com 40 ou mais anos é de {value_mean_days_employed_gender_m_age_over40:.2f}.')
print()

#Valor médio das mulheres com que possuem filhos
value_mean_days_employed_gender_f_child = new_table.loc[(new_table['gender'] == 'F')\
                                          &(new_table['children']>0)]['days_employed'].mean()
print(f'Valor médio de dias trabalhados das mulheres que possuem filho(s) é de {value_mean_days_employed_gender_f_child:.2f}.')

#Valor médio dos homens com que possuem filhos
value_mean_days_employed_gender_m_child = new_table.loc[(new_table['gender'] == 'M')\
                                           &(new_table['children']>0)]['days_employed'].mean()
print(f'Valor médio de dias trabalhados dos homens que possuem filho(s) é de {value_mean_days_employed_gender_m_child:.2f}.')
print()

#Valor médio das mulheres com curso superior/formadas
value_mean_days_employed_gender_f_f = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')]['days_employed'].mean()
print(f'Valor médio de dias trabalhados das mulheres formadas é de {value_mean_days_employed_gender_f_f:.2f}.')

#Valor médio dos homens com curso superior/formados
value_mean_days_employed_gender_m_f = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')]['days_employed'].mean()
print(f'Valor médio de dias trabalhados dos homens formados é de {value_mean_days_employed_gender_m_f:.2f}.')
print()

#Valor médio das mulheres com curso superior/formadas e que possuem filhos
value_mean_days_employed_gender_f_f_child = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')\
                                       &(new_table['children']>0)]['days_employed'].mean()
print(f'Valor médio de dias trabalhados das mulheres formadas que possuem filho(s) é de {value_mean_days_employed_gender_f_f_child:.2f}.')

#Valor médio dos homens com curso superior/formados e que possuem filhos
value_mean_days_employed_gender_m_f_child = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')\
                                       &(new_table['children']>0)]['days_employed'].mean()
print(f'Valor médio de dias trablhados dos homens formados que possuem filho(s) é de {value_mean_days_employed_gender_m_f_child:.2f}.')
print()

#Valor médio das mulheres com curso superior/formadas, com menos de 40 anos e que possuem filhos
value_mean_days_employed_gender_f_f_child_age_under40 = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='F')\
                                       &(new_table['dob_years']<40)\
                                       &(new_table['children']>0)]['days_employed'].mean()
print(f'Valor médio de dias trablhados das mulheres formadas que possuem filho(s) com menos de 40 anos é de {value_mean_days_employed_gender_f_f_child_age_under40:.2f}.')

#Valor médio dos homens com curso superior/formados, com menos de 40 anos e que possuem filhos
value_mean_days_employed_gender_m_f_child_age_under40 = new_table.loc[(new_table['education'] == 'bachelor\'s degree')\
                                       &(new_table['gender']=='M')\
                                       &(new_table['dob_years']<40)\
                                       &(new_table['children']>0)]['days_employed'].mean()
print(f'Valor médio de dias trablhados dos homens formados que possuem filho(s) com menos de 40 anos é de {value_mean_days_employed_gender_m_f_child_age_under40:.2f}.')
print()

Valor médio de dias trabalhados de todos os valores é de 5962.87.

Valor médio de dias trabalhados das mulheres é de 6945.66.
Valor médio de dias trabalhados dos homens é de 4063.26.

Valor médio de dias trabalhados das mulheres com menos de 40 anos é de 1866.71.
Valor médio de dias trabalhados dos homens com menos de 40 anos é de 1770.16.
Valor médio de dias trabalhados das mulheres com 40 ou mais anos é de 9990.61.
Valor médio de dias trabalhados dos homens com 40 ou mais anos é de 6239.84.

Valor médio de dias trabalhados das mulheres que possuem filho(s) é de 3254.89.
Valor médio de dias trabalhados dos homens que possuem filho(s) é de 2704.90.

Valor médio de dias trabalhados das mulheres formadas é de 4894.34.
Valor médio de dias trabalhados dos homens formados é de 3790.82.

Valor médio de dias trabalhados das mulheres formadas que possuem filho(s) é de 2736.23.
Valor médio de dias trablhados dos homens formados que possuem filho(s) é de 2666.39.

Valor médio de dias trablhados 

Diante das análises realizadas, valores ausentes serão subistiuídos por medianas da seguintes formas:
- para pessoas que possuem nível superior (formadas), os valores ausentes serão preenchidos pela mediana referente à dias trabalhados por gênero e formados.
- para pessoas que não possuem nível superior, os valores ausentes serão preenchidos pela mediana referente referente à dias trabalhados por gênero.

In [49]:
#Definindo uma função para preencher os valores ausentes.
def fill_missing_values_days_employed(row):
    days_employed = row['days_employed']
    gender = row['gender']
    education_id = row['education_id']
    
    if days_employed > 0:
        return days_employed
    if education_id == 0:
        if gender == 'M':
            return value_median_days_employed_gender_m_f
        else:
            return value_median_days_employed_gender_f_f
    if education_id > 0:
        if gender == 'M':
            return value_median_days_employed_gender_m
        else:
            return value_median_days_employed_gender_f


In [50]:
# Verificando se a função funciona
row_values2 = [[0, 'M', 3000], [0, 'F', 7700], [10,'F',0],[0, 'M', 0], [0, 'F', 0],[10, 'M', 0]]
row_columns2 = ['education_id', 'gender', 'days_employed']
df_teste2 = pd.DataFrame(row_values2, columns=row_columns2)
print(df_teste2.head(10))
print()

try:
    df_teste2['days_employed'] = df_teste2.apply(fill_missing_values_days_employed, axis=1)
except:
    'Erro de execução!'
    
print(df_teste2)


   education_id gender  days_employed
0             0      M           3000
1             0      F           7700
2            10      F              0
3             0      M              0
4             0      F              0
5            10      M              0

   education_id gender  days_employed
0             0      M    3000.000000
1             0      F    7700.000000
2            10      F    2539.761232
3             0      M    1659.309688
4             0      F    2011.171809
5            10      M    1662.370103


In [51]:
# Aplicando a função na coluna 'days_employed'.

default_risk['days_employed'] = default_risk.apply(fill_missing_values_days_employed, axis=1)

In [52]:
# Verificando se a função funcionou.
print(default_risk['days_employed'].isna().sum())
print()
print(default_risk.loc[default_risk['days_employed']==0])
print()
print(default_risk.loc[default_risk['days_employed']<0])
print()
display(default_risk.head(60))



0

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,buy or renovate residential properties
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,buy a car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,buy or renovate residential properties
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education
4,0,22630.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,buy or renovate residential properties
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,buy real estate (undefined)
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,buy or renovate residential properties


In [53]:
# Verificando o número de entradas na coluna. Se 'days_employed' realmente foi preenchida.

default_risk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21273 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21273 non-null  int64  
 1   days_employed     21273 non-null  float64
 2   dob_years         21273 non-null  int64  
 3   education         21273 non-null  object 
 4   education_id      21273 non-null  int64  
 5   family_status     21273 non-null  object 
 6   family_status_id  21273 non-null  int64  
 7   gender            21273 non-null  object 
 8   income_type       21273 non-null  object 
 9   debt              21273 non-null  int64  
 10  total_income      21273 non-null  float64
 11  purpose           21273 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


## Categorização de dados

Após pré processar os dados e deixá-los mais organizados e limpos, será realizada a categorização de algumas informações para que possamos ter informações que auxiliará na tomada de decisões e para responder as hipóteses levantadas

Neste primeiro momento iremos categorizar por faixa de idade:
- de 10 à 19 anos;
- de 20 à 29 anos;
- de 30 à 39 anos;
- de 40 à 49 anos;
- de 50 à 59 anos;
- de 60 à 69 anos;
- 70 ou mais anos.

In [54]:
# Definindo uma função que calcule a categoria de idade.
def calc_age(age):
    if age < 20:
        return '10-19'
    elif age < 30:
        return '20-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    elif age < 70:
        return '60-69'
    else:
        return '70+'


In [55]:
# Testando se a função funciona

age_test0 = 15
age_test1 = 25
age_test2 = 35
age_test3 = 45
age_test4 = 55
age_test5 = 65
age_test6 = 75

print(calc_age(age_test0))
print(calc_age(age_test1))
print(calc_age(age_test2))
print(calc_age(age_test3))
print(calc_age(age_test4))
print(calc_age(age_test5))
print(calc_age(age_test6))



10-19
20-29
30-39
40-49
50-59
60-69
70+


In [56]:
# Criando uma coluna nova com base na função chamada 'age_category'.
default_risk['age_category'] = default_risk['dob_years'].apply(calc_age)


In [57]:
# Verificando se ocorreu tudo certo.

display(default_risk.head(60))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,buy or renovate residential properties,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,buy a car,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,buy or renovate residential properties,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,30-39
4,0,22630.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50-59
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,buy or renovate residential properties,20-29
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,buy real estate (undefined),40-49
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,wedding,30-39
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,buy or renovate residential properties,40-49


Agora vamos categorizar a renda total das pessoas ('total_income')

In [58]:
# Examinando dados numéricos da coluna 'total_income' para categorização
display(default_risk['total_income'].head(60))
print()
display(default_risk['total_income'].tail(60))

0      40620.102
1      17932.802
2      23341.752
3      42820.568
4      25378.572
5      40922.170
6      38484.156
7      21731.829
8      15337.093
9      23108.150
10     18230.959
11     12331.077
12     26834.295
13     20873.317
14     26420.466
15     18691.345
16     46272.433
17     14465.694
18      9091.804
19     38852.977
20     33528.423
21     21089.953
22     23948.983
23     20522.515
24     46487.558
25      8818.041
26     26834.295
27     49415.837
28     30058.118
29     21465.165
30     27432.971
31     44077.710
32     22249.194
33     25159.326
34     16745.672
35     12448.908
36     22212.904
37     24660.621
38     30759.568
39    120678.528
40     22858.493
41     21465.165
42     13130.414
43     43673.141
44     16124.879
45     17021.747
46     29229.194
47     57004.465
48     25930.483
49      7134.689
50     14774.837
51     32511.949
52      8439.428
53     49832.576
54     14758.210
55     21465.165
56     23862.567
57      9378.625
58     66304.6

21465    27523.7500
21466    11765.7520
21467     7582.2300
21468    26260.4440
21469    28608.8560
21470    33680.7950
21471    39059.7510
21472    58030.6290
21473    32035.8130
21474    45264.3570
21475     7718.2900
21476    84392.4530
21477    11477.4250
21478    28989.7380
21479    27440.6960
21480    23511.2070
21481    70475.3410
21482    15050.4050
21483    12386.8100
21484    15306.7920
21485    42720.1170
21486    12922.2170
21487    14053.5300
21488    20623.5980
21489    26834.2950
21490     5562.8740
21491    24883.3440
21492    47237.4740
21493    23600.4160
21494    28219.1350
21495    21465.1650
21496    19102.8190
21497    26063.4715
21498    38522.8120
21499    25208.5050
21500    12450.1270
21501    13797.1400
21502    21465.1650
21503    42280.1600
21504    12890.6110
21505    12070.3990
21506    23286.7190
21507    15708.8450
21508    11622.1750
21509    11684.6500
21510    21465.1650
21511    22410.9560
21512    23568.2330
21513    40157.7830
21514    56958.1450


In [59]:
# Obtendo mais informações da coluna 'total_income'.
print(default_risk['total_income'].min())
print()
print(default_risk['total_income'].max())
print()
print(default_risk['total_income'].mean())
print()
print(default_risk['total_income'].median())
print()

3306.762

362496.645

26591.288944789172

23246.394



Vamos categorizar essas informações de renda em 6 (seis) níveis, são eles:
- Nível 1: renda de 0 à 19999
- Nível 2: renda de 20000 à 39999
- Nível 3: renda de 40000 à 59999
- Nível 4: renda de 60000 à 79999
- Nível 5: renda de 80000 à 99999
- Nível 6: renda igual ou superior 100000

Foram definidas essas faixas de níveis por se tratar do quanto as pessoas possuem de renda. Com essas informações facilitarão as verificações das possibilidades de inadiplências dos indivíduos. 

In [60]:
# Definindo uma função que calcule a categoria de renda.

def risk_level(income):
    if income < 20000:
        return 'Nível 1'
    elif income < 40000:
        return 'Nível 2'
    elif income < 60000:
        return 'Nível 3'
    elif income < 80000:
        return 'Nível 4'
    elif income < 100000:
        return 'Nível 5'
    else:
        return 'Nível 6'

In [61]:
# Criando coluna 'risk_level' com categorias das rendas.
try:
    default_risk['risk_level'] = default_risk['total_income'].apply(risk_level)
except:
    'Erro de execução!'

# Verificando se deu certo.
display(default_risk.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,risk_level
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,buy or renovate residential properties,40-49,Nível 3
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,buy a car,30-39,Nível 1
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,buy or renovate residential properties,30-39,Nível 2
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,education,30-39,Nível 3
4,0,22630.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,wedding,50-59,Nível 2


In [62]:
# Contando os valores de cada categoria para ver a distribuição
display(default_risk['risk_level'].value_counts())


Nível 2    11092
Nível 1     7369
Nível 3     2140
Nível 4      450
Nível 5      123
Nível 6       99
Name: risk_level, dtype: int64

## Verificar as Hipóteses


**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [63]:
# Verifique os dados do nível de renda e do pagamento em dia
print(default_risk['debt'].value_counts())

risk_level_debt = default_risk.pivot_table(index='risk_level', values='debt', aggfunc='sum', margins=True)

# Calcular a taxa de inadimplência com base no nível de renda
total_default = default_risk['debt'].sum()
def tx_default(debt):
    if debt > 0:
        return debt/total_default
    
try:
    risk_level_debt['default_rate'] = risk_level_debt['debt'].apply(tx_default)
except:
    'Erro de execução!'

display(risk_level_debt)

0    19533
1     1740
Name: debt, dtype: int64


,debt,default_rate
risk_level,,
Nível 1,608,0.349425
Nível 2,938,0.539080
Nível 3,156,0.089655
Nível 4,24,0.013793
Nível 5,8,0.004598
Nível 6,6,0.003448
All,1740,1.000000


**Conclusão**

Ao analisando os dados, considerando o nível de renda com os pagamentos em dia, percebemos que há uma correlação entre eles.

A maioria da inadimplência se concentram nos níveis de renda 1 e 2.

Estes 2 níveis de renda são responsáveis por **88,85%** das inadimplências registradas e individualmente são:
- Nível 1 = **34,94%**
- Nível 2 = **53,91%**

As pessoas com as menores rendas são responsáveis por **88,85%** das inadimplências.

**Existe uma correlação entre o status familiar e o pagamento em dia?**

In [64]:
# Verifique os dados de status da família e do pagamento em dia
risk_family_status_debt = default_risk.pivot_table(index='family_status', values='debt', aggfunc='sum', margins=True)

# Calcular a taxa padrão com base no status da família
total_default = default_risk['debt'].sum()
def tx_family_status_default(debt):
    if debt > 0:
        return debt/total_default
    
try:
    risk_family_status_debt['tx_family_status_default'] = risk_family_status_debt['debt'].apply(tx_family_status_default)
except:
    'Erro de execução!'

display(risk_family_status_debt)


,debt,tx_family_status_default
family_status,,
civil partnership,388,0.222989
divorced,85,0.048851
married,930,0.534483
unmarried,274,0.157471
widow/widower,63,0.036207
All,1740,1.000000


**Conclusão**

Quando analisando os dados referentes ao pagamento das dívidas em dia com relação ao status familar, percebemos que as pessoas cassadas são responsáveis por **53,45%** das inadimplências registradas.

As pessoas com união estável (civil partnership) são responsáveis por uma fatia de **22,30%** das inadiplências.

Com isso percebemos que pessoas que vivem com um conjuge são responsáveis por **75,75%** do total de inadimplência.

**Existe uma correlação entre ter filhos ou não e o pagamento em dia?**

In [65]:
# Verifique os dados das crianças e do pagamento em dia
risk_children_debt = default_risk.pivot_table(index='children', values='debt', aggfunc='sum', margins=True)


# Calcular a taxa de inadimplência com base no número de filhos
total_default = default_risk['debt'].sum()
def tx_children_default(debt):
    if debt > 0:
        return debt/total_default
    
try:
    risk_children_debt['tx_children_default'] = risk_children_debt['debt'].apply(tx_children_default)
except:
    'Erro de execução!'

display(risk_children_debt)



,debt,tx_children_default
children,,
0,1062,0.610345
1,445,0.255747
2,202,0.116092
3,27,0.015517
4,4,0.002299
5,0,NaN
All,1740,1.000000


**Conclusão**

A correlação entre ter filhos ou não e o pagamento em dia se confirma, pois entre as pessoas inadimplentes os que não possuem filhos são responsáveis por **61,03%** dos atrasos nos pagamentos. Seguido, com **25,57%**, pelas pessoas com 1 filho apenas e logo após os com 2 filhos com **11,61%** dos registros.

Estes 3 grupos são responsáveis por **98,21%** do total.

**Existe uma correlação com a finalidade do crédito afeta a taxa de inadimplência?**

In [66]:
# Confira os percentuais de inadimplência para cada finalidade de crédito e analise-os
risk_purpose_debt = default_risk.pivot_table(index='purpose', values='debt', aggfunc='sum', margins=True)


# Calcular a taxa de inadimplência com base na finalidade do crédito
total_default = default_risk['debt'].sum()
def tx_purpose_default(debt):
    if debt > 0:
        return debt/total_default
    
try:
    risk_purpose_debt['tx_purpose_default'] = risk_purpose_debt['debt'].apply(tx_purpose_default)
except:
    'Erro de execução!'

display(risk_purpose_debt)

,debt,tx_purpose_default
purpose,,
building a property (undefined),102,0.058621
buy a car,402,0.231034
buy commercial real estate,99,0.056897
buy or renovate residential properties,299,0.171839
buy real estate (undefined),240,0.137931
construction of own property,42,0.024138
education,370,0.212644
wedding,186,0.106897
All,1740,1.000000


**Conclusão**

A análise sobre as finalidades do crétidos nos mostra que este parâmetro afeta sim a taxa de inadimplência.

A compra de carro é o principal ofensor com **23,10%** do total de caso seguido pela educação com **21,26%**. Estes representam **44,36%** das inadimplências.

# Conclusão Geral 

Haviam no banco de dados informações inconsistentes que tiveram de ser tratadas.

Valores ausentes foram preenchidos com medianas, após análises de dados realizadas definindo qual parâmetro seria mais adequado.

Inconsistências em informações nas diversas colunas foram avaliados e corrigidos, após estudo para as ações a serem tomadas.

Duplicatas foram excluídas por ser improvável se tratar de indivíduos distintos. Foram um total de 252 linhas deletadas do banco de dados. Este volume não impactou nos resultados finais. 

As categorizações realizadas foram importantes para uma visualização melhor das informações para a tomada de decisões da companhia.

As 4 (quatro) hipóteses testadas/analisadas se confirmaram relação com taxas de inadimplências.